<a href="https://colab.research.google.com/github/lkarjun/Data-Science-from-Scratch/blob/master/23%20Recommender%20Systems/recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recommending system

In [1]:
users_interests = [["Hadoop", "Big Data", "HBase", "Java", "Spark","Storm", "Cassandra"],
                   ["NoSQL", "MongoDB", "Cassandra", "HBase","Postgres"],
                   ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
                   ["R", "Python", "statistics", "regression", "probability"],
                   ["machine learning", "regression", "decision trees", "libsvm"],
                   ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
                   ["statistics", "probability", "mathematics", "theory"],
                   ["machine learning", "scikit-learn", "Mahout", "neural networks"],
                   ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
                   ["Hadoop", "Java", "MapReduce", "Big Data"],
                   ["statistics", "R", "statsmodels"],
                   ["C++", "deep learning", "artificial intelligence", "probability"],
                   ["pandas", "R", "Python"],
                   ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
                   ["libsvm", "regression", "support vector machines"]]

### Recommending what's popular

In [2]:
from collections import Counter

In [3]:
popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests)

In [4]:
from typing import List, Tuple

In [5]:
def most_popular_new_interests(
                user_interests: List[str],
                max_results: int = 5) -> List[Tuple[str, int]]:

                suggestions = [(interest, frequency)
                                for interest, frequency in popular_interests.most_common()
                                if interest not in user_interests]

                return suggestions[:max_results]


In [6]:
most_popular_new_interests(users_interests[1])

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

### User-Based Collabrative Filtering

In [7]:
unique_interests = sorted({interest for user_interests in users_interests
                           for interest in user_interests})

In [8]:
unique_interests[:4]

['Big Data', 'C++', 'Cassandra', 'HBase']

In [15]:
def make_user_interest_vector(user_interests: List[str]) -> List[int]:
  '''given a list of interests, produce a vector whose ith element is 1
     if unique_interests[i] is in the list, 0 otherwise'''
  return [1 if interest in user_interests else 0
             for interest in unique_interests]